In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import regexp_replace
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/Projeto/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [6]:
hdfs_path = "hdfs://hdfs-nn:9000/Projeto/demo/bronze/Draft/1976_to_2015_Drafts.csv"
hdfs_path_auxiliar = "hdfs://hdfs-nn:9000/Projeto/demo/bronze/Champions/Nba_Champions.csv"

In [7]:
# Create a DataFrame from JSON data (automatically infer schema and data types)
# There are other file formats you can read from (e.g., csv, orc, parquet)
# https://spark.apache.org/docs/2.2.0/sql-programming-guide.html#data-sources

# Read Sillicon valley episodes data
dataFrame = spark.read.csv(hdfs_path, header= True)
dataFrame_auxiliar = spark.read.csv(hdfs_path_auxiliar, header= True)

In [8]:
dataFrame.toPandas()

22/01/13 23:24:37 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Player, All_NBA, All.Star, Draft_Yr, Pk, Team, College, Yrs, Games, Minutes.Played, PTS, TRB, AST, FG_Percentage, TP_Percentage, FT_Percentage, Minutes.per.Game, Points.per.Game, TRB.per.game, Assits.per.Game, Win.Share, WS_per_game, BPM, VORP, Executive, Tenure, Exec_ID, Exec_draft_exp, attend_college, first_year, second_year, third_year, fourth_year, fifth_year
 Schema: _c0, Player, All_NBA, All.Star, Draft_Yr, Pk, Team, College, Yrs, Games, Minutes.Played, PTS, TRB, AST, FG_Percentage, TP_Percentage, FT_Percentage, Minutes.per.Game, Points.per.Game, TRB.per.game, Assits.per.Game, Win.Share, WS_per_game, BPM, VORP, Executive, Tenure, Exec_ID, Exec_draft_exp, attend_college, first_year, second_year, third_year, fourth_year, fifth_year
Expected: _c0 but found: 
CSV file: hdfs://hdfs-nn:9000/Projeto/demo/bronze/Draft/1976_to_2015_Drafts.csv


_c0           Player All_NBA All.Star Draft_Yr   Pk Team  \
0        1    Robert Parish       2        9     1976    8  GSW   
1        2     Sonny Parker       0        0     1976   17  GSW   
2        3  Marshall Rogers       0        0     1976   34  GSW   
3        4      Jeff Fosnes       0        0     1976   68  GSW   
4        5        Carl Bird       0        0     1976   86  GSW   
...    ...              ...     ...      ...      ...  ...  ...   
3956  3957        Ken Owens       0        0     1982   88  OKC   
3957  3958         Rod Camp       0        0     1982  111  OKC   
3958  3959      Bobby Potts       0        0     1982  134  OKC   
3959  3960    Allen Rayhorn       0        0     1982  157  OKC   
3960  3961      Steve Burks       0        0     1982  180  OKC   

                                        College Yrs Games  ...  \
0                Centenary College of Louisiana  21  1611  ...   
1                          Texas A&M University   6   452  ...   
2              University of Texas-Pan American   1    26  ...   
3                         Vanderbilt University   0     0  ...   
4                      University of California   0     0  ...   
...                                         ...  ..   ...  ...   
3956                        University of Idaho   0     0  ...   
3957               Southern Illinois University   0     0  ...   
3958  University of North Carolina at Charlotte   0     0  ...   
3959               Northern Illinois University   0     0  ...   
3960                   University of Washington   0     0  ...   

           Executive                        Tenure Exec_ID Exec_draft_exp  \
0          Al Attles  3641 days 00:00:00.000000000       1              1   
1          Al Attles  3641 days 00:00:00.000000000       1              1   
2          Al Attles  3641 days 00:00:00.000000000       1              1   
3          Al Attles  3641 days 00:00:00.000000000       1              1   
4          Al Attles  3641 days 00:00:00.000000000       1              1   
...              ...                           ...     ...            ...   
3956  Zollie Volchok  2189 days 00:00:00.000000000     162              6   
3957  Zollie Volchok  2189 days 00:00:00.000000000     162              6   
3958  Zollie Volchok  2189 days 00:00:00.000000000     162              6   
3959  Zollie Volchok  2189 days 00:00:00.000000000     162              6   
3960  Zollie Volchok  2189 days 00:00:00.000000000     162              6   

     attend_college first_year second_year third_year fourth_year fifth_year  
0                 1          0           0          0           0          0  
1                 1          0           0          0           0          0  
2                 1          0           0          0           0          0  
3                 1          0           0          0           0          0  
4                 1          0           0          0           0          0  
...             ...        ...         ...        ...         ...        ...  
3956              1          0           0          0           0          0  
3957              1          0           0          0           0          0  
3958              1          0           0          0           0          0  
3959              1          0           0          0           0          0  
3960              1          0           0          0           0          0  

[3961 rows x 35 columns]

In [9]:
dfa_auxiliar = dataFrame_auxiliar.withColumnRenamed("_c0","Year")\
             .withColumnRenamed("_c1", "League")\
            .withColumnRenamed("Finals2", "Champion")\
            .withColumnRenamed("Finals3", "Runner-Up")\
            .withColumnRenamed("Finals4", "Finals MVP")\
            .withColumnRenamed("_c5", "Eliminate")\
            .withColumnRenamed("Playoffs Top Performers6", "Points")\
            .withColumnRenamed("Playoffs Top Performers7", "Rebounds")\
            .withColumnRenamed("Playoffs Top Performers8", "Assists")\
            .withColumnRenamed("Playoffs Top Performers9", "Win Shares")
#dfa_auxiliar.toPandas()          

In [10]:
filtered_auxiliar = dfa_auxiliar.filter (dfa_auxiliar.Year != "Year")
#filtered_auxiliar.toPandas()

In [11]:
drop_auxiliar= filtered_auxiliar.drop('Eliminate')

In [12]:
drop_main= dataFrame.drop('FG_Percentage')
drop_main= drop_main.drop('TP_Percentage')
drop_main= drop_main.drop('FT_Percentage')
drop_main= drop_main.drop('Minutes.per.Game')
drop_main= drop_main.drop('Points.per.Game')
drop_main= drop_main.drop('TRB.per.game')
drop_main= drop_main.drop('Win.Share')
drop_main= drop_main.drop('Assits.per.Game')
drop_main= drop_main.drop('WS_per_game')
drop_main= drop_main.drop('BPM')
drop_main= drop_main.drop('VORP')
drop_main= drop_main.drop('Executive')
drop_main= drop_main.drop('Tenure')
drop_main= drop_main.drop('Exec_ID')
drop_main= drop_main.drop('Exec_draft_exp')
drop_main= drop_main.drop('attend_college')
drop_main= drop_main.drop('first_year')
drop_main= drop_main.drop('second_year')
drop_main= drop_main.drop('third_year')
drop_main= drop_main.drop('fourth_year')
drop_main= drop_main.drop('fifth_year')
drop_main= drop_main.drop('_c0')
drop_main= drop_main.drop('College')
drop_main= drop_main.drop('TRB')

#drop_main.toPandas()


In [13]:
finished_auxiliar = drop_auxiliar.filter((drop_auxiliar.League != "ABA"))
finished_auxiliar = finished_auxiliar.filter((drop_auxiliar.League != "BAA"))
finished_auxiliar.toPandas()

22/01/13 23:24:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , Finals, Finals, Finals, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers
 Schema: _c0, _c1, Finals2, Finals3, Finals4, Playoffs Top Performers6, Playoffs Top Performers7, Playoffs Top Performers8, Playoffs Top Performers9
Expected: _c0 but found: 
CSV file: hdfs://hdfs-nn:9000/Projeto/demo/bronze/Champions/Nba_Champions.csv


Year League               Champion              Runner-Up  \
0   2021    NBA        Milwaukee Bucks           Phoenix Suns   
1   2020    NBA     Los Angeles Lakers             Miami Heat   
2   2019    NBA        Toronto Raptors  Golden State Warriors   
3   2018    NBA  Golden State Warriors    Cleveland Cavaliers   
4   2017    NBA  Golden State Warriors    Cleveland Cavaliers   
..   ...    ...                    ...                    ...   
67  1954    NBA     Minneapolis Lakers     Syracuse Nationals   
68  1953    NBA     Minneapolis Lakers        New York Knicks   
69  1952    NBA     Minneapolis Lakers        New York Knicks   
70  1951    NBA       Rochester Royals        New York Knicks   
71  1950    NBA     Minneapolis Lakers     Syracuse Nationals   

          Finals MVP                  Points                Rebounds  \
0   G. Antetokounmpo  G. Antetokounmpo (634)  G. Antetokounmpo (269)   
1           L. James          A. Davis (582)          L. James (226)   
2         K. Leonard        K. Leonard (732)          D. Green (223)   
3          K. Durant          L. James (748)          D. Green (222)   
4          K. Durant          L. James (591)           K. Love (191)   
..               ...                     ...                     ...   
67              None          G. Mikan (252)          G. Mikan (171)   
68              None          G. Mikan (238)          G. Mikan (185)   
69              None          G. Mikan (307)          G. Mikan (207)   
70              None          A. Risen (273)          A. Risen (196)   
71              None          G. Mikan (376)                    None   

             Assists              Win Shares  
0   J. Holiday (199)  G. Antetokounmpo (3.7)  
1     L. James (184)          A. Davis (4.5)  
2     D. Green (187)        K. Leonard (4.9)  
3     L. James (198)          L. James (5.2)  
4     L. James (141)          L. James (4.3)  
..               ...                     ...  
67    S. Martin (60)          G. Mikan (3.5)  
68   D. McGuire (70)          G. Mikan (2.0)  
69   D. McGuire (90)          G. Mikan (2.4)  
70   D. McGuire (78)         B. Wanzer (2.7)  
71   J. Pollard (56)          G. Mikan (3.7)  

[72 rows x 9 columns]

In [14]:
finished_auxiliar = finished_auxiliar.withColumnRenamed("Finals MVP","Finals_MVP")\
             .withColumnRenamed("Win Shares", "Win_Shares")\
            .withColumnRenamed("Runner-Up", "Runner_Up")

In [15]:
finished_auxiliar.show(3000)

+----+------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+
|Year|League|            Champion|           Runner_Up|      Finals_MVP|              Points|            Rebounds|             Assists|          Win_Shares|
+----+------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+
|2021|   NBA|     Milwaukee Bucks|        Phoenix Suns|G. Antetokounmpo|G. Antetokounmpo ...|G. Antetokounmpo ...|    J. Holiday (199)|G. Antetokounmpo ...|
|2020|   NBA|  Los Angeles Lakers|          Miami Heat|        L. James|      A. Davis (582)|      L. James (226)|      L. James (184)|      A. Davis (4.5)|
|2019|   NBA|     Toronto Raptors|Golden State Warr...|      K. Leonard|    K. Leonard (732)|      D. Green (223)|      D. Green (187)|    K. Leonard (4.9)|
|2018|   NBA|Golden State Warr...| Cleveland Cavaliers|   

22/01/13 23:24:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , Finals, Finals, Finals, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers
 Schema: _c0, _c1, Finals2, Finals3, Finals4, Playoffs Top Performers6, Playoffs Top Performers7, Playoffs Top Performers8, Playoffs Top Performers9
Expected: _c0 but found: 
CSV file: hdfs://hdfs-nn:9000/Projeto/demo/bronze/Champions/Nba_Champions.csv


In [16]:
finished_auxiliar=finished_auxiliar.withColumn('Champion', regexp_replace('Champion', 'Minneapolis Lakers', 'Los Angeles Lakers'))
finished_auxiliar=finished_auxiliar.withColumn('Champion', regexp_replace('Champion', 'Philadelphia Warriors', 'Golden State Warriors'))
finished_auxiliar=finished_auxiliar.withColumn('Champion', regexp_replace('Champion', 'St. Louis Hawks', 'Atlanta Hawks'))
finished_auxiliar=finished_auxiliar.withColumn('Champion', regexp_replace('Champion', 'Washington Bullets', 'Washington Wizards'))
finished_auxiliar=finished_auxiliar.withColumn('Champion', regexp_replace('Champion', 'Seattle SuperSonics', 'Oklahoma City Thunder'))
finished_auxiliar=finished_auxiliar.withColumn('Champion', regexp_replace('Champion', 'Rochester Royals', 'Sacramento Kings'))
finished_auxiliar=finished_auxiliar.withColumn('Runner_Up', regexp_replace('Runner_Up', 'Fort Wayne Pistons', 'Detroit Pistons'))
finished_auxiliar=finished_auxiliar.withColumn('Runner_Up', regexp_replace('Runner_Up', 'St. Louis Hawks', 'Atlanta Hawks'))
finished_auxiliar=finished_auxiliar.withColumn('Runner_Up', regexp_replace('Runner_Up', 'Minneapolis Lakers', 'Los Angeles Lakers'))
finished_auxiliar=finished_auxiliar.withColumn('Runner_Up', regexp_replace('Runner_Up', 'San Francisco Warriors', 'Golden State Warriors'))
finished_auxiliar=finished_auxiliar.withColumn('Runner_Up', regexp_replace('Runner_Up', 'Washington Bullets', 'Washington Wizards'))
finished_auxiliar=finished_auxiliar.withColumn('Runner_Up', regexp_replace('Runner_Up', 'New Jersey Nets', 'Brooklyn Nets'))
finished_auxiliar=finished_auxiliar.withColumn('Runner_Up', regexp_replace('Runner_Up', 'Seattle SuperSonics', 'Oklahoma City Thunder'))
finished_auxiliar=finished_auxiliar.withColumn('Runner_Up', regexp_replace('Runner_Up', 'Baltimore Bullets', 'Oklahoma City Thunder'))

In [17]:
finished_auxiliar.show(72)

+----+------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+
|Year|League|            Champion|           Runner_Up|      Finals_MVP|              Points|            Rebounds|             Assists|          Win_Shares|
+----+------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+
|2021|   NBA|     Milwaukee Bucks|        Phoenix Suns|G. Antetokounmpo|G. Antetokounmpo ...|G. Antetokounmpo ...|    J. Holiday (199)|G. Antetokounmpo ...|
|2020|   NBA|  Los Angeles Lakers|          Miami Heat|        L. James|      A. Davis (582)|      L. James (226)|      L. James (184)|      A. Davis (4.5)|
|2019|   NBA|     Toronto Raptors|Golden State Warr...|      K. Leonard|    K. Leonard (732)|      D. Green (223)|      D. Green (187)|    K. Leonard (4.9)|
|2018|   NBA|Golden State Warr...| Cleveland Cavaliers|   

22/01/13 23:24:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , Finals, Finals, Finals, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers
 Schema: _c0, _c1, Finals2, Finals3, Finals4, Playoffs Top Performers6, Playoffs Top Performers7, Playoffs Top Performers8, Playoffs Top Performers9
Expected: _c0 but found: 
CSV file: hdfs://hdfs-nn:9000/Projeto/demo/bronze/Champions/Nba_Champions.csv


In [18]:
drop_main = drop_main.withColumnRenamed("All.Star","All_Star")\
             .withColumnRenamed("Minutes.Played", "Minutes_Played")

In [19]:
from pyspark.sql.types import DoubleType
from pyspark.sql.types import *

finished_auxiliar = finished_auxiliar.withColumn("Year", finished_auxiliar["Year"].cast(IntegerType())) 


In [20]:
from pyspark.sql.types import DoubleType
from pyspark.sql.types import *

drop_main = drop_main.withColumn("All_NBA", drop_main["All_NBA"].cast(IntegerType())) 
drop_main = drop_main.withColumn("All_Star", drop_main["All_Star"].cast(IntegerType()))
drop_main = drop_main.withColumn("Draft_Yr", drop_main["Draft_Yr"].cast(IntegerType())) 
drop_main = drop_main.withColumn("Pk", drop_main["Pk"].cast(IntegerType()))
drop_main = drop_main.withColumn("Yrs", drop_main["Yrs"].cast(IntegerType())) 
drop_main = drop_main.withColumn("Games", drop_main["Games"].cast(IntegerType())) 
drop_main = drop_main.withColumn("Minutes_Played",drop_main["Minutes_Played"].cast(IntegerType())) 
drop_main = drop_main.withColumn("PTS", drop_main["PTS"].cast(IntegerType())) 
drop_main = drop_main.withColumn("AST", drop_main["AST"].cast(IntegerType())) 




In [21]:

finished_auxiliar = finished_auxiliar.filter((drop_auxiliar.Champion != "Syracuse Nationals"))
finished_auxiliar.show()

+----+------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+--------------------+
|Year|League|            Champion|           Runner_Up|      Finals_MVP|              Points|            Rebounds|           Assists|          Win_Shares|
+----+------+--------------------+--------------------+----------------+--------------------+--------------------+------------------+--------------------+
|2021|   NBA|     Milwaukee Bucks|        Phoenix Suns|G. Antetokounmpo|G. Antetokounmpo ...|G. Antetokounmpo ...|  J. Holiday (199)|G. Antetokounmpo ...|
|2020|   NBA|  Los Angeles Lakers|          Miami Heat|        L. James|      A. Davis (582)|      L. James (226)|    L. James (184)|      A. Davis (4.5)|
|2019|   NBA|     Toronto Raptors|Golden State Warr...|      K. Leonard|    K. Leonard (732)|      D. Green (223)|    D. Green (187)|    K. Leonard (4.9)|
|2018|   NBA|Golden State Warr...| Cleveland Cavaliers|       K. Duran

22/01/13 23:25:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , Finals, Finals, Finals, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers
 Schema: _c0, _c1, Finals2, Finals3, Finals4, Playoffs Top Performers6, Playoffs Top Performers7, Playoffs Top Performers8, Playoffs Top Performers9
Expected: _c0 but found: 
CSV file: hdfs://hdfs-nn:9000/Projeto/demo/bronze/Champions/Nba_Champions.csv


In [22]:
drop_main.createOrReplaceTempView("drop_main")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM drop_main
    WHERE Draft_Yr=="2005"
    """
)
sqlized_df.show(3000)

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

+--------------------+-------+--------+--------+---+----+---+-----+--------------+-----+----+
|              Player|All_NBA|All_Star|Draft_Yr| Pk|Team|Yrs|Games|Minutes_Played|  PTS| AST|
+--------------------+-------+--------+--------+---+----+---+-----+--------------+-----+----+
|          Chris Paul|      8|       9|    2005|  4| NOH| 11|  773|         27725|14506|7688|
|        Brandon Bass|      0|       0|    2005| 33| NOH| 11|  706|         15833| 6283| 587|
|      Raymond Felton|      0|       0|    2005|  5| CHA| 11|  776|         25385| 9608|4650|
|            Sean May|      0|       0|    2005| 13| CHA|  4|  119|          1868|  821| 115|
|      Louis Williams|      0|       0|    2005| 45| PHI| 11|  701|         16442| 8604|2036|
|     Marvin Williams|      0|       0|    2005|  2| ATL| 11|  785|         22596| 8273|1019|
|    Salim Stoudamire|      0|       0|    2005| 31| ATL|  3|  157|          2672| 1260| 164|
|          Cenk Akyol|      0|       0|    2005| 59| ATL|  0

In [31]:
drop_main.printSchema()

root
 |-- Player: string (nullable = true)
 |-- All_NBA: integer (nullable = true)
 |-- All_Star: integer (nullable = true)
 |-- Draft_Yr: integer (nullable = true)
 |-- Pk: integer (nullable = true)
 |-- Team: string (nullable = true)
 |-- Yrs: integer (nullable = true)
 |-- Games: integer (nullable = true)
 |-- Minutes_Played: integer (nullable = true)
 |-- PTS: integer (nullable = true)
 |-- AST: integer (nullable = true)



In [23]:
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'GSW', 'Golden State Warriors'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'LAL', 'Los Angeles Lakers'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'SAS', 'San Antonio Spurs'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'WAS', 'Washington Wizards'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'DEN', 'Denver Nuggets'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'NJN', 'Brooklyn Nets'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'IND', 'Indiana Pacers'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'MIL', 'Milwaukee Bucks'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'NYK', 'New York Knicks'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'UTA', 'Utah Jazz'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'POR', 'Portland Trail Blazers'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'DET', 'Detroit Pistons'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'PHO', 'Phoenix Suns'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'SAC', 'Sacramento Kings'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'LAC', 'Los Angeles Clippers'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'DAL', 'Dallas Mavericks'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'PHI', 'Philadelphia 76ers'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'HOU', 'Houston Rockets'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'BOS', 'Boston Celtics'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'CHI', 'Chicago Bulls'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'CLE', 'Cleveland Cavaliers'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'ATL', 'Atlanta Hawks'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'OKC', 'Oklahoma City Thunder'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'NOH', 'New Orleans Pelicans'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'MIA', 'Miami Heat'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'MIN', 'Minnesota Timberwolves'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'ORL', 'Orlando Magic'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'TOR', 'Toronto Raptors'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'MEM', 'Memphis Grizzlies'))
drop_main=drop_main.withColumn('Team', regexp_replace('Team', 'CHA', 'Charlotte Hornets'))









In [24]:
drop_main.createOrReplaceTempView("drop_main")

# Then you can write all the SQL you want, as your json is seen as a regular table now.
sqlized_df = spark.sql(
    """
    SELECT *
    FROM drop_main
    WHERE Draft_Yr=="2000"
    """
)
sqlized_df.show(3000)

+------------------+-------+--------+--------+---+--------------------+---+-----+--------------+-----+----+
|            Player|All_NBA|All_Star|Draft_Yr| Pk|                Team|Yrs|Games|Minutes_Played|  PTS| AST|
+------------------+-------+--------+--------+---+--------------------+---+-----+--------------+-----+----+
|    Speedy Claxton|      0|       0|    2000| 20|  Philadelphia 76ers|  7|  334|          8548| 3096|1441|
|      Mark Karcher|      0|       0|    2000| 48|  Philadelphia 76ers|  0|    0|             0|    0|   0|
|    Stromile Swift|      0|       0|    2000|  2|   Memphis Grizzlies|  9|  547|         10804| 4582| 275|
|   Jamaal Magloire|      0|       0|    2000| 19|New Orleans Pelicans| 12|  680|         14621| 4917| 386|
|     Erick Barkley|      0|       0|    2000| 28|Portland Trail Bl...|  2|   27|           266|   77|  40|
|   Jake Tsakalidis|      0|       0|    2000| 25|        Phoenix Suns|  7|  315|          5005| 1497| 101|
|    Joel Przybilla|      0|

In [25]:
auxiliar_nomes=drop_main.select("Team")
auxiliar_nomes=auxiliar_nomes.distinct()
auxiliar_nomes.toPandas()


Team
0             Phoenix Suns
1           Boston Celtics
2         Dallas Mavericks
3     New Orleans Pelicans
4            Brooklyn Nets
5          New York Knicks
6        Memphis Grizzlies
7   Minnesota Timberwolves
8       Los Angeles Lakers
9    Golden State Warriors
10           Orlando Magic
11    Los Angeles Clippers
12         Detroit Pistons
13           Chicago Bulls
14         Milwaukee Bucks
15     Cleveland Cavaliers
16          Indiana Pacers
17      Washington Wizards
18           Atlanta Hawks
19               Utah Jazz
20       Charlotte Hornets
21      Philadelphia 76ers
22  Portland Trail Blazers
23         Toronto Raptors
24       San Antonio Spurs
25   Oklahoma City Thunder
26        Sacramento Kings
27         Houston Rockets
28              Miami Heat
29          Denver Nuggets

In [26]:
cond = [finished_auxiliar.Champion == auxiliar_nomes.Team]
df_junto=finished_auxiliar.join(auxiliar_nomes,cond, 'outer')

In [27]:
df_junto.show(3000)

22/01/13 23:25:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , Finals, Finals, Finals, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers
 Schema: _c0, _c1, Finals2, Finals3, Finals4, Playoffs Top Performers6, Playoffs Top Performers7, Playoffs Top Performers8, Playoffs Top Performers9
Expected: _c0 but found: 
CSV file: hdfs://hdfs-nn:9000/Projeto/demo/bronze/Champions/Nba_Champions.csv


+----+------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Year|League|            Champion|           Runner_Up|      Finals_MVP|              Points|            Rebounds|             Assists|          Win_Shares|                Team|
+----+------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|null|  null|                null|                null|            null|                null|                null|                null|                null|        Phoenix Suns|
|2008|   NBA|      Boston Celtics|  Los Angeles Lakers|       P. Pierce|     K. Bryant (633)|    K. Garnett (274)|      R. Rondo (172)|    K. Garnett (4.1)|      Boston Celtics|
|1986|   NBA|      Boston Celtics|     Houston Rockets|         L. Bird|   H. Olajuwon (537)|   H. Olajuwon (2

In [28]:
finished_main = drop_main

In [29]:
finished_main \
    .select("*") \
    .write \
    .mode("overwrite") \
    .partitionBy("Team") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .save("hdfs://hdfs-nn:9000/Projeto/NBA.db/sl_Draft_picks")

In [30]:
df_junto \
    .select("*") \
    .write \
    .mode("overwrite") \
    .partitionBy("Team") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .save("hdfs://hdfs-nn:9000/Projeto/NBA.db/sl_Champions")

22/01/13 23:26:28 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , Finals, Finals, Finals, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers, Playoffs Top Performers
 Schema: _c0, _c1, Finals2, Finals3, Finals4, Playoffs Top Performers6, Playoffs Top Performers7, Playoffs Top Performers8, Playoffs Top Performers9
Expected: _c0 but found: 
CSV file: hdfs://hdfs-nn:9000/Projeto/demo/bronze/Champions/Nba_Champions.csv


In [ ]:
finished_auxiliar.show(3000)